# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
M. Hobson  ->  M. Hobson  |  ['M. Hobson']
F. Walter  ->  F. Walter  |  ['F. Walter']
A. Müller  ->  A. Müller  |  ['A. Müller']


J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 76 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2309.15905


extracting tarball to tmp_2309.15905...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:473: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

extracting tarball to tmp_2309.16294...

 done.
Retrieving document from  https://arxiv.org/e-print/2309.16528


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2309.16294/AugerOpenData.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'acknowledgments' from 'tmp_2309.16294/acknowledgments.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'latex_authorlist_institutions' from 'tmp_2309.16294/latex_authorlist_institutions.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcac

extracting tarball to tmp_2309.16528...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.15905-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.15905) | **Wide post-common envelope binaries containing ultramassive white dwarfs:  evidence for efficient envelope ejection in massive AGB stars**  |
|| N. Yamaguchi, et al. -- incl., <mark>K. El-Badry</mark>, <mark>M. Hobson</mark> |
|*Appeared on*| *2023-09-29*|
|*Comments*| *19 pages, 9 figures, submitted to MNRAS*|
|**Abstract**| Post-common-envelope binaries (PCEBs) containing a white dwarf (WD) and a main-sequence (MS) star can constrain the physics of common envelope evolution and calibrate binary evolution models. Most PCEBs studied to date have short orbital periods ($P_{\rm orb} \lesssim 1$ d), implying relatively inefficient harnessing of binaries' orbital energy for envelope expulsion. Here, we present follow-up observations of five binaries from {\it Gaia} DR3 containing solar-type MS stars and probable ultramassive WDs ($M\gtrsim 1.2\,M_{\odot}$) with significantly wider orbits than previously known PCEBs, $P_{\rm orb} = 18-49$ d. The WD masses are much higher than expected for systems formed via stable mass transfer at these periods, and their near-circular orbits suggest partial tidal circularization when the WD progenitors were giants. These properties strongly suggest that the binaries are PCEBs. Forming PCEBs at such wide separations requires highly efficient envelope ejection, and we find that the observed periods can only be explained if a significant fraction of the energy released when the envelope recombines goes into ejecting it. Using 1D stellar evolution calculations, we show that the binding energy of massive AGB star envelopes is formally {\it positive} if recombination energy is included in the energy budget. This suggests that the star's envelope can be efficiently ejected if binary interaction causes it to recombine, and that a wide range of PCEB orbital periods can potentially result from Roche lobe overflow of an AGB star. This evolutionary scenario may also explain the formation of several wide WD+MS binaries discovered via self-lensing. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.16528-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.16528) | **A new discovery space opened by eROSITA: Ionised AGN outflows from X-ray  selected samples**  |
|| B. Musiimenta, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2023-09-29*|
|*Comments*| *Accepted for publication in Astronomy and Astrophysics(A&A), 24 pages, 18 figures, 5 tables*|
|**Abstract**| In the context of an evolutionary model, the outflow phase of an Active Galactic Nuclei (AGN) occurs at the peak of its activity, once the central SMBH is massive enough to generate sufficient power to counterbalance the potential well of the host galaxy. This phase plays a vital role in galaxy evolution. We aim to apply various selection methods to isolate powerful AGNs in the feedback phase, trace and characterise their outflows, and explore the link between AGN luminosity and outflow properties. We applied a combination of methods to the eROSITA Final Equatorial Depth survey (eFEDS) catalogue and isolated ~1400 candidates at z>0.5 out of ~11750 AGNs (~12\%). We tested the robustness of our selection on the small subsample of 50 sources with available good quality SDSS spectra at 0.5<z<1, for which we fitted the [OIII] emission line complex and searched for the presence of ionised gas outflows. We identified 23 quasars (~45\%) with evidence of ionised outflows based on the presence of significant broad and shifted components in the [OIII] line. They are on average more luminous and more obscured than the parent sample, although this may be ascribed to selection effects affecting the good quality SDSS spectra sample. By adding 118 outflowing quasars at 0.5<z<3.5 from the literature, we find a weak correlation between the maximum outflow velocity and AGN bolometric luminosity. On the contrary, we find strong correlations between mass outflow rate and outflow kinetic power with the AGN bolometric luminosity. About 30\% of our sample have kinetic coupling efficiencies >1\%. We find that the majority of the outflows have momentum flux ratios lower than 20 which rules out an energy-conserving nature. Our present work points to the unequivocal existence of a rather short AGN outflow phase, paving the way towards a new avenue to dissect AGN outflows in large samples within eROSITA and beyond. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.15906-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.15906) | **[CII] Spectral Mapping of the Galactic Wind and Starbursting Disk of M82  with SOFIA**  |
|| R. C. Levy, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2023-09-29*|
|*Comments*| *22 pages, 9 figures, Accepted for publication in ApJ*|
|**Abstract**| M82 is an archetypal starburst galaxy in the local Universe. The central burst of star formation, thought to be triggered by M82's interaction with other members in the M81 group, is driving a multiphase galaxy-scale wind away from the plane of the disk that has been studied across the electromagnetic spectrum. Here, we present new velocity-resolved observations of the [CII] 158$\mu$m line in the central disk and the southern outflow of M82 using the upGREAT instrument onboard SOFIA. We also report the first detections of velocity-resolved ($\Delta V = 10$ km s$^{-1}$) [CII] emission in the outflow of M82 at projected distances of $\approx1-2$ kpc south of the galaxy center. We compare the [CII] line profiles to observations of CO and HI and find that likely the majority ($>55$%) of the [CII] emission in the outflow is associated with the neutral atomic medium. We find that the fraction of [CII] actually outflowing from M82 is small compared to the bulk gas outside the midplane (which may be in a halo or tidal streamers), which has important implications for observations of [CII] outflows at higher redshift. Finally, by comparing the observed ratio of the [CII] and CO intensities to models of photodissociation regions, we estimate that the far-ultraviolet (FUV) radiation field in the disk is $\sim10^{3.5}~G_0$, in agreement with previous estimates. In the outflow, however, the FUV radiation field is 2-3 orders of magnitudes lower, which may explain the high fraction of [CII] arising from the neutral medium in the wind. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.15934-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.15934) | **The effect of cluster dynamical state on ram-pressure stripping**  |
|| A. Lourenço, et al. -- incl., <mark>A. Müller</mark> |
|*Appeared on*| *2023-09-29*|
|*Comments*| *18 pages, 8 figures. Accepted for publication in MNRAS*|
|**Abstract**| Theoretical and observational studies have suggested that ram-pressure stripping by the intracluster medium can be enhanced during cluster interactions, boosting the formation of the "jellyfish" galaxies. In this work, we study the incidence of galaxies undergoing ram-pressure stripping in 52 clusters of different dynamical states. We use optical data from the WINGS/OmegaWINGS surveys and archival X-ray data to characterise the dynamical state of our cluster sample, applying eight different proxies. We then compute the number of ram-pressure stripping candidates relative to the infalling population of blue late-type galaxies within a fixed circular aperture in each cluster. We find no clear correlation between the fractions of ram-pressure stripping candidates and the different cluster dynamical state proxies considered. These fractions also show no apparent correlation with cluster mass. To construct a dynamical state classification closer to a merging "sequence", we perform a visual classification of the dynamical states of the clusters, combining information available in optical, X-ray, and radio wavelengths. We find a mild increase in the RPS fraction in interacting clusters with respect to all other classes (including post-mergers). This mild enhancement could hint at a short-lived enhanced ram-pressure stripping in ongoing cluster mergers. However, our results are not statistically significant due to the low galaxy numbers. We note this is the first homogeneous attempt to quantify the effect of cluster dynamical state on ram-pressure stripping using a large cluster sample, but even larger (especially wider) multi-wavelength surveys are needed to confirm the results. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.16294-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.16294) | **The Pierre Auger Observatory Open Data**  |
|| P. A. Collaboration, et al. |
|*Appeared on*| *2023-09-29*|
|*Comments*| *19 pages, 8 figures*|
|**Abstract**| The Pierre Auger Collaboration has embraced the concept of open access to their research data since its foundation, with the aim of giving access to the widest possible community. A gradual process of release began as early as 2007 when 1% of the cosmic-ray data was made public, along with 100% of the space-weather information. In February 2021, a portal was released containing 10% of cosmic-ray data collected from 2004 to 2018, during Phase I of the Observatory. The Portal included detailed documentation about the detection and reconstruction procedures, analysis codes that can be easily used and modified and, additionally, visualization tools. Since then the Portal has been updated and extended. In 2023, a catalog of the 100 highest-energy cosmic-ray events examined in depth has been included. A specific section dedicated to educational use has been developed with the expectation that these data will be explored by a wide and diverse community including professional and citizen-scientists, and used for educational and outreach initiatives. This paper describes the context, the spirit and the technical implementation of the release of data by the largest cosmic-ray detector ever built, and anticipates its future developments. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2309.15905.md
    + _build/html/tmp_2309.15905/./Figures/MESA/HR_Diagram.jpeg
    + _build/html/tmp_2309.15905/./Figures/MESA/a_f_plots_a_i.jpeg
    + _build/html/tmp_2309.15905/./Figures/avMwd.jpeg
    + _build/html/tmp_2309.15905/./Figures/PorbvEcc.jpeg


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\@}{tempa}$
$\newcommand{\@}{tempa }$
$\newcommand{\@}{tempb }$
$\newcommand{\@}{tempc$
$  }$
$\newcommand{\@}{tempb }$</div>



<div id="title">

# Wide post-common envelope binaries containing ultramassive white dwarfs: evidence for efficient envelope ejection in massive AGB stars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.15905-b31b1b.svg)](https://arxiv.org/abs/2309.15905)<mark>Appeared on: 2023-09-29</mark> -  _19 pages, 9 figures, submitted to MNRAS_

</div>
<div id="authors">

N. Yamaguchi, et al. -- incl., <mark>K. El-Badry</mark>, <mark>M. Hobson</mark>

</div>
<div id="abstract">

**Abstract:** Post-common-envelope binaries (PCEBs) containing a white dwarf (WD) and a main-sequence (MS) star can constrain the physics of common envelope evolution and calibrate binary evolution models. Most PCEBs studied to date have short orbital periods ( $P_{\rm orb} \lesssim 1$ d), implying relatively inefficient harnessing of binaries' orbital energy for envelope expulsion. Here, we present follow-up observations of five binaries from $_ Gaia_$ DR3 containing solar-type MS stars and probable ultramassive WDs ( $M\gtrsim 1.2 M_{\odot}$ ) with significantly wider orbits than previously known PCEBs, $P_{\rm orb} = 18-49$ d. The WD masses are much higher than expected for systems formed via stable mass transfer at these periods, and their near-circular orbits suggest partial tidal circularization when the WD progenitors were giants. These properties strongly suggest that the binaries are PCEBs. Forming PCEBs at such wide separations requires highly efficient envelope ejection, and we find that the observed periods can only be explained if a significant fraction of the energy released when the envelope recombines goes into ejecting it. Using 1D stellar evolution calculations, we show that the binding energy of massive AGB star envelopes is formally $_ positive_$ if recombination energy is included in the energy budget. This suggests that the star's envelope can be efficiently ejected if binary interaction causes it to recombine, and that a wide range of PCEB orbital periods can potentially result from Roche lobe overflow of an AGB star. This evolutionary scenario may also explain the formation of several wide WD+MS binaries discovered via self-lensing.

</div>

<div id="div_fig1">

<img src="tmp_2309.15905/./Figures/MESA/HR_Diagram.jpeg" alt="Fig7.1" width="50%"/><img src="tmp_2309.15905/./Figures/MESA/a_f_plots_a_i.jpeg" alt="Fig7.2" width="50%"/>

**Figure 7. -** _Left_: HR diagram showing the evolution of a $7 M_{\odot}$ star starting from pre-MS to the AGB. The blue sections indicate what we refer to as the RGB (but which also includes the SGB) and the red sections represents the AGB. _Center_: Plots of the final separation $a_f$(i.e. birth period at the end of CEE) over a range of initial separations $a_i$. $a_i$ is taken to be the orbital semi-major axis when the giant (WD progenitor) fills its Roche lobe. We mark $a_f =$ 0.01 AU $\sim 2 R_{\odot}$(red dashed line) below which the MS star would not fit in the orbit and a PCEB cannot form. The orange dashed line is the case where only the gravitational binding energy is considered and $\alpha_{\rm CE} = 1$. We see that in this case, no values of $a_i$ result in $a_f > 0.15$ AU (gray dashed line) which is approximately the minimum separation of our objects. The other three lines are the cases where internal energy is added to the binding energy for $\alpha_{\rm CE} = 0.3$, 0.6, and 0.9. We see that these lines lie above the dashed line for some range of $a_i$. _Right_: Zoom in on the region where $a_f > 0.15$ AU. We see that overall, $a_i \sim 3.5 - 4.4$ AU can result in the wide orbital separations of our systems. (*fig:mesa_af*)

</div>
<div id="div_fig2">

<img src="tmp_2309.15905/./Figures/avMwd.jpeg" alt="Fig6" width="100%"/>

**Figure 6. -** Periastron separation, $a_{\rm peri}$, vs. WD mass, $M_{\rm WD}$, for a sample of PCEBs from \citet[][circle markers]{Zorotovic2011A&A} and the five objects from this work (stars; the arrows indicate lower limits). IK Peg is distinguished from the other known PCEBs with a triangle marker as it lies very close to our objects. We also plot the PCEBs from the "white dwarf binary pathways survey" \citep[diamond markers;][]{Hernandez2021MNRAS, Hernandez2022MNRAS_vi, Hernandez2022MNRAS}. Finally, we plot self-lensing binaries (SLBs) discovered by \citet{Kawahara2018AJ} as well as  KOI-3278 \citep[][also a SLB]{Kruse2014Sci}, which were all detected by _ Kepler_. The colours of the points represents the masses of the luminous MS companions. The dashed gray shows the prediction for stable MT from \citet{Rappaport1995MNRAS} and the blue line indicates the maximum radius reached by the WD progenitor for a range of WD masses.  (*fig:avMwd*)

</div>
<div id="div_fig3">

<img src="tmp_2309.15905/./Figures/PorbvEcc.jpeg" alt="Fig1" width="100%"/>

**Figure 1. -** Eccentricity vs. orbital period. We plot the sample of MSP + WD binaries mainly from the ATNF catalogue (\citealt{Manchester2005AJ}, with a few others, compiled by \citealt{Hui2018ApJ}), differentiating between those with minimum WD mass above and below $0.45 M_{\odot}$(orange triangles and blue circles, respectively). We show our objects with magenta stars. The gray line is the theoretical relation for MSP + He WD binaries formed through stable MT derived by \citet{Phinney1992RSPTA}. The points circled in red are eMSPs \citep[e.g., see ][for a description of the five plotted here]{Stovall2019ApJ}. We also circle in green two binaries with massive CO WDs with large eccentricities \citep{Lorimer2015MNRAS, Berezina2017MNRAS}. (*fig:PvEcc*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.15905"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

283  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
